# CAO Points Notebook
[http://www.cao.ie/index.php?page=points&p=2021](http://www.cao.ie/index.php?page=points&p=2021)
***

In [1]:
# Convenient HTTP requests
import requests as rq # Requests
# Regular Expressions
import re # Regular Expressions
# Dates and times
import datetime as dt

In [2]:
# Fetch the CAO URL
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Testing this request
resp

<Response [200]>

In [3]:
# Get the current date and time
now = dt.datetime.now()
# Save the date and time as a string so it can be reused
now_str = now.strftime('%Y%m%d_%H%M%S')
# Create a file path for the original data using the date and time
path = 'cao2021_' + now_str + '.html'

In [4]:
# Server uses the wrong encoding, we need to change it from iso-8859-1 to cp1252
original_encoding = resp.encoding
original_encoding
resp.encoding = 'cp1252'

In [5]:
# Save the file
with open(path, 'w') as f:
    f.write(resp.text)

In [6]:
# Compile the regular expression for matching lines with course info
# re_course = re.compile('([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *') # Ian's code
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)   +(.*)  +(.*)')

In [7]:
# Loop through the lines of the response content
no_lines = 0
for line in resp.iter_lines():
    if re_course.match(line.decode('iso-8859-1')):
        #print(line)
        no_lines = no_lines +1
      
# Check how many lines are being captured
print (no_lines)

949


In [8]:
csv_path = 'cao2021_re_' + now_str + '.csv'
with open(csv_path,'w') as f:
    
    # Now loop through and print out the lines matching the RE
    for line in resp.iter_lines():
        if re_course.match(line.decode('cp1252')):
            csv_version = re_course.sub(r'\1,\2,\3,\4', line.decode('cp1252'))
            #print(csv_version)
            #print(line)
            f.write(csv_version + '\n')
            
            # Using the line split method
            # Split the lines on two or more spaces
            #linesplit = re.split('  +',line.decode('cp1252'))
            #print(','.join(linesplit))
            #f.write(','.join(linesplit) + '\n') 